In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_csv('data.csv')
data.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                            int64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [8]:
features = ['hotel', 'lead_time', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'arrival_date_day_of_month', 'adults', 'children', 'country']
target = 'is_canceled'

X = data[features]
y = data[target]

label_encoders = {}
for column in X.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    X.loc[:, column] = label_encoders[column].fit_transform(X.loc[:, column].values.reshape(-1, 1)).copy()
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
X_test = X_test.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

params_dt = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

decision_tree = DecisionTreeClassifier(random_state=42)
grid_search_dt = GridSearchCV(decision_tree, params_dt, cv=5)
grid_search_dt.fit(X_train_scaled, y_train)

best_decision_tree = grid_search_dt.best_estimator_
predictions_dt_tuned = best_decision_tree.predict(X_test_scaled)
accuracy_dt_tuned = accuracy_score(y_test, predictions_dt_tuned)

print("Лучшие параметры для решающего дерева:", grid_search_dt.best_params_)
print("Точность настроенных параметров решающего дерева:", accuracy_dt_tuned)
print("Classification Report of Tuned Decision Tree:")
print(classification_report(y_test, predictions_dt_tuned))

params_rf = {
    'n_estimators': [50, 100],
    'max_depth': [None, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1]
}

random_forest = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(random_forest, params_rf, cv=5)
grid_search_rf.fit(X_train_scaled, y_train)

best_random_forest = grid_search_rf.best_estimator_
predictions_rf_tuned = best_random_forest.predict(X_test_scaled)
accuracy_rf_tuned = accuracy_score(y_test, predictions_rf_tuned)

print("\nЛушчие параметры для случайного леса:", grid_search_rf.best_params_)
print("Точность настроенных параметров случайного леса:", accuracy_rf_tuned)
print("Classification Report of Tuned Random Forest:")
print(classification_report(y_test, predictions_rf_tuned))

D:\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ivan5\AppData\Local\Temp\ipykernel_2052\2342474418.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, column] = label_encoders[column].fit_transform(X.loc[:, column].values.reshape(-1, 1)).copy()
C:\Users\ivan5\AppData\Local\Temp\ipykernel_2052\2342474418.py:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or

Лучшие параметры для решающего дерева: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}
Точность настроенных параметров решающего дерева: 0.8443291326908822
Classification Report of Tuned Decision Tree:
              precision    recall  f1-score   support

           0       0.85      0.93      0.89      4486
           1       0.84      0.66      0.74      2259

    accuracy                           0.84      6745
   macro avg       0.84      0.80      0.81      6745
weighted avg       0.84      0.84      0.84      6745


Лушчие параметры для случайного леса: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Точность настроенных параметров случайного леса: 0.8704225352112676
Classification Report of Tuned Random Forest:
              precision    recall  f1-score   support

           0       0.89      0.92      0.90      4486
           1       0.83      0.77      0.80      2259

    accuracy                           0.87     

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Считывание данных
df = pd.read_csv('data.csv')

# Вывод информации о данных
print("Информация о данных:")
print(f"Количество строк: {df.shape[0]}")
print(f"Количество столбцов: {df.shape[1]}")
print("\nТипы данных по столбцам:")
print(df.dtypes)

Информация о данных:
Количество строк: 33725
Количество столбцов: 32

Типы данных по столбцам:
hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                            int64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes        

In [ ]:
# Определение целевой переменной и признаков
X = df.drop('is_canceled', axis=1)
y = df['is_canceled']
X = pd.get_dummies(X)

# Преобразование категориальных признаков в числовые
label_encoder = LabelEncoder()
for column in X.columns:
    if X[column].dtype == 'object':
        X[column] = label_encoder.fit_transform(X[column])
        
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Замена отсутсвующих и бесконечных значений на ноль
X_train = X_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
X_test = X_test.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Определение параметров для поиска
param_grid = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.1],
    'depth': [3, 6],
    'loss_function': ['MultiClass']
}

# Создание объекта GridSearchCV для модели
grid_search = GridSearchCV(
    estimator=CatBoostClassifier(verbose=False),  # Установка verbose=False,чтобы не было видно как обучается модель 
    param_grid=param_grid,
    cv=5,
    n_jobs=-1
)


# Подгонка модели к данным
grid_search.fit(X_train_scaled, y_train)

# Получение лучших параметров и оценщика с лучшими параметрами
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f'Лучшие параметры: {best_params}')

# Получение прогнозов с использованием лучшей модели
y_pred = best_model.predict(X_test_scaled)

# Вычисление и вывод точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

# Оценка производительности модели
report = classification_report(y_test, y_pred, zero_division=1)
print(f'Отчет о классификации:\n{report}')

# Определение отсутствующих классов в прогнозах
classes = np.unique(y_test)
predicted_classes = np.unique(y_pred)